In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install imbalanced-learn
!pip install ppscore

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ppscore as pps

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder, LabelBinarizer
from sklearn.metrics import confusion_matrix, fbeta_score, accuracy_score, roc_curve, roc_auc_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from imblearn.over_sampling import RandomOverSampler 
from imblearn.pipeline import Pipeline, make_pipeline

import statsmodels.api as sm
import statsmodels.formula.api as smf

## Pre-Processing

In [ ]:
## load the Spillover dataset
spillover_path = "drive/MyDrive/Senior Thesis Data/Spillover_dataset_Sept_22_2020.csv"
spillover_data = pd.read_csv(spillover_path)

In [ ]:
## preprocess the Spillover dataset

# select the covariates of interest
spillover_data = spillover_data[["HostOrder", "HostFamily", "HostGenus", "VirusGenus", "VirusFamily", "Genome_General", "Segmented", "Envelope", "SeverityDiseaseHumans", "ChronicityHumans", "Total_transmission", "HumanToHumanTransVirus"]]

# remove duplicate rows
spillover_data = spillover_data.drop_duplicates()

# remove rows with missing data
spillover_data = spillover_data[spillover_data.VirusGenus != "Unassigned"]
spillover_data = spillover_data[spillover_data.SeverityDiseaseHumans != "SickUnknownCause"]
spillover_data = spillover_data[spillover_data.ChronicityHumans != "Unknown"]

In [ ]:
## because the dataset is unbalanced (some categorical levels only appear once), to avoid the curse of dimensionality after one-hot encoding collapse categorical feature levels that appear >5 times in the dataset
spillover_data.loc[spillover_data['HostGenus'].value_counts()[spillover_data['HostGenus']].values < 5, 'HostGenus'] = "Other" # (reference: https://stackoverflow.com/questions/22208562/replace-rarely-occurring-values-in-a-pandas-dataframe)

spillover_data.loc[spillover_data['HostOrder'].value_counts()[spillover_data['HostOrder']].values < 5, 'HostOrder'] = "Other"

spillover_data.loc[spillover_data['HostFamily'].value_counts()[spillover_data['HostFamily']].values < 5, 'HostFamily'] = "Other"

spillover_data.loc[spillover_data['VirusGenus'].value_counts()[spillover_data['VirusGenus']].values < 5, 'VirusGenus'] = "Other"

spillover_data.loc[spillover_data['VirusFamily'].value_counts()[spillover_data['VirusFamily']].values < 5, 'VirusFamily'] = "Other"

In [ ]:
# pull out the descriptive and target features
X = spillover_data.iloc[:, :-1]
y = spillover_data.iloc[:, -1]

# binarize the target feature
lb = LabelBinarizer()
y = lb.fit_transform(y) ## 1 is for Yes, 0 is for No

## one-hot encode the categorical features 
X= pd.get_dummies(X, columns=["HostOrder", "HostFamily", "HostGenus", "VirusGenus", "VirusFamily", "Genome_General", "Segmented", "Envelope", "SeverityDiseaseHumans", "ChronicityHumans"])

## split data into train and test sets
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size= 0.3, train_size= 0.7, random_state=10)

# balance the training dataset by oversampling the minority class
oversample = RandomOverSampler(sampling_strategy='minority')
X_train, y_train = oversample.fit_resample(X_train, y_train)

## Exploratory Data Analysis (EDA)

In [ ]:
## use a contingency table to look at the associations between descriptive features and the target feature
host_genus_trans = pd.DataFrame(pd.crosstab(spillover_data['HostGenus'], spillover_data['HumanToHumanTransVirus'], margins = False))
host_genus_trans = host_genus_trans.sort_values(by=['Yes'], ascending=False)
print(host_genus_trans.head(n=5))

host_order_trans = pd.DataFrame(pd.crosstab(spillover_data['HostOrder'], spillover_data['HumanToHumanTransVirus'], margins = False))
host_order_trans = host_order_trans.sort_values(by=['Yes'], ascending=False)
print(host_order_trans.head(n=5))

host_family_trans = pd.DataFrame(pd.crosstab(spillover_data['HostFamily'], spillover_data['HumanToHumanTransVirus'], margins = False))
host_family_trans = host_family_trans.sort_values(by=['Yes'], ascending=False)
print(host_family_trans.head(n=5))

genome_trans = pd.DataFrame(pd.crosstab(spillover_data['Genome_General'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(genome_trans) 

segmented_trans = pd.DataFrame(pd.crosstab(spillover_data['Segmented'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(segmented_trans) ## segmentation: when it works, it works (check to see if those 10 are like flu viruses)

envelope_trans = pd.DataFrame(pd.crosstab(spillover_data['Envelope'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(envelope_trans) ## most of the H2H viruses are enveloped, but a higher proportion of the non-enveloped viruses are H2H

virus_genus_trans = pd.DataFrame(pd.crosstab(spillover_data['VirusGenus'], spillover_data['HumanToHumanTransVirus'], margins = False))
virus_genus_trans = virus_genus_trans.sort_values(by=['Yes'], ascending=False)
print(virus_genus_trans.head(n=5))

virus_family_trans = pd.DataFrame(pd.crosstab(spillover_data['VirusFamily'], spillover_data['HumanToHumanTransVirus'], margins = False))
virus_family_trans = virus_family_trans.sort_values(by=['Yes'], ascending=False)
print(virus_family_trans) ## some of the findings of this are consistent with the graph here: https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1006215

severity_trans = pd.DataFrame(pd.crosstab(spillover_data['SeverityDiseaseHumans'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(severity_trans)

chronicity_trans = pd.DataFrame(pd.crosstab(spillover_data['ChronicityHumans'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(chronicity_trans)

transmission_trans = pd.DataFrame(pd.crosstab(spillover_data['Total_transmission'], spillover_data['HumanToHumanTransVirus'], margins = False))
print(transmission_trans)

In [ ]:
## visualize the contingency tables with stacked bar plots
transmission_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of Transmission Modes")
plt.xlabel("Number of Transmission Modes")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
chronicity_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of Chronicity")
plt.xlabel("Chronicity")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
severity_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of Severity")
plt.xlabel("Severity")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
envelope_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of Viral Envelope")
plt.xlabel("Presence of Viral Envelope")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
segmented_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of RNA Segmentation")
plt.xlabel("Presence of Segmentation")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
genome_trans.plot(kind='bar', stacked=True)
plt.title("Distribution of Genome Type")
plt.xlabel("Genome Type")
plt.ylabel("Count")
plt.xticks(rotation=360)
plt.legend(title="H2H Virus")

In [ ]:
## look at which descriptive features are most predictive of the target variable
pps.predictors(spillover_data, 'HumanToHumanTransVirus', output='df', sorted= True)[['x', 'ppscore']]

In [ ]:
## perform feature selection based on chi-squared scores
select = SelectKBest(score_func= chi2, k=10)
best = select.fit_transform(X, y)

names = select.get_support()
all_features = np.array(X.columns)
best = all_features[names]

print(best)

## Model Selection

### Information-Based Classification Models

In [ ]:
## use GridSearchCV to tune model hyperparameters (reference: https://towardsdatascience.com/gridsearchcv-for-beginners-db48a90114ee; https://towardsdatascience.com/imbalanced-class-sizes-and-classification-models-a-cautionary-tale-part-2-cf371500d1b3)
dt_pipeline = Pipeline([('sampling', RandomOverSampler()), ('class', DecisionTreeClassifier(random_state=10))])

params = {'class__criterion': ['gini', 'entropy'],
        'class__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}

gs_decision_tree = GridSearchCV(dt_pipeline, params, scoring='f1', cv= 10) 
gs_decision_tree.fit(X_train, y_train)
print("Best Parameters:", gs_decision_tree.best_params_)

## train a decision tree model on the data
decision_tree = DecisionTreeClassifier(random_state= 10, criterion='entropy', max_depth=9) 
decision_tree = decision_tree.fit(X_train, y_train)

y_train_pred = decision_tree.predict(X_train)
y_test_pred= decision_tree.predict(X_test)

## print the confusion matrices for the training and testing datasets
print("Decision Tree Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("Decision Tree Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the accuracy score of the model on the training and testing datasets
print("Training Accuracy:", fbeta_score(y_train, y_train_pred, beta=1))
print("Testing Accuracy:", fbeta_score(y_test, y_test_pred, beta=1)) 

Decision Tree Training Dataset Confusion Matrix
 [[248   8]
 [  0 256]]
Decision Tree Testing Dataset Confusion Matrix
 [[101   4]
 [  1  21]]
Training Accuracy: 0.9846153846153847
Testing Accuracy: 0.8936170212765958


In [ ]:
## use GridSearchCV to tune model hyperparameters 
rf_pipeline = Pipeline([('sampling', RandomOverSampler()), ('class', RandomForestClassifier())])

params = {'class__criterion': ['gini', 'entropy'],
        'class__n_estimators': [10, 15, 25, 50, 100, 150, 200],
        'class__max_depth': [5, 10, 15]}

rf_gs = GridSearchCV(rf_pipeline, params, scoring='f1', cv= 10)
rf_gs.fit(X_train, y_train)
print("Best Parameters:", rf_gs.best_params_)

## train a Random Forest classifier on the data
random_forest = RandomForestClassifier(random_state= 10, n_estimators= 100, max_depth= 15, criterion= 'gini')
random_forest = random_forest.fit(X_train, y_train)

y_train_pred = random_forest.predict(X_train)
y_test_pred = random_forest.predict(X_test)

print("Random Forest Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("Random Forest Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the accuracy score of the model on the training and testing datasets
print("Training Accuracy:", fbeta_score(y_train, y_train_pred, beta=1))
print("Testing Accuracy:", fbeta_score(y_test, y_test_pred, beta=1)) 

Random Forest Training Dataset Confusion Matrix
 [[246   7]
 [  0 253]]
Random Forest Testing Dataset Confusion Matrix
 [[107   1]
 [  1  18]]
Training Accuracy: 0.98635477582846
Testing Accuracy: 0.9473684210526315


### Similarity-Based Classification Models

In [ ]:
## use GridSearchCV to tune model hyperparameters 
knn_pipeline = Pipeline([('sampling', RandomOverSampler()), ('class', KNeighborsClassifier())])

params = {'class__n_neighbors': [1, 3, 5, 7, 9],
           'class__weights': ['uniform', 'distance']}

gs_knn = GridSearchCV(knn_pipeline, params, scoring='f1', cv= 5)
gs_knn.fit(X_train, y_train)
print("Best Parameters:", gs_knn.best_params_)

## train a Nearest Neighbor binary classification model on the data
knn = KNeighborsClassifier(n_neighbors = 1, weights= 'uniform')
knn = knn.fit(X_train, y_train)

y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

## display the confusion matrix 
print("\nK Nearest Neighbor Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("K Nearest Neighbor Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the accuracy score of the model on the training and testing datasets
print("\nTraining Accuracy:", fbeta_score(y_train, y_train_pred, beta=1))
print("Testing Accuracy:", fbeta_score(y_test, y_test_pred, beta=1)) 


K Nearest Neighbor Training Dataset Confusion Matrix
 [[256   0]
 [ 18 238]]
K Nearest Neighbor Testing Dataset Confusion Matrix
 [[103   2]
 [  2  20]]

Training Accuracy: 0.9635627530364372
Testing Accuracy: 0.9090909090909091


### Probability-Based Classification Models

In [ ]:
## create a Naive Bayes classifier
naive_bayes = BernoulliNB()
naive_bayes = naive_bayes.fit(X_train, y_train)

y_train_pred = naive_bayes.predict(X_train)
y_test_pred = naive_bayes.predict(X_test)

## display the confusion matrix 
print("Naive Bayes Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("Naive Bayes Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the F2 score for the training and testing datasets
print("Training Accuracy:", fbeta_score(y_train, y_train_pred, beta=1))
print("Testing Accuracy:", fbeta_score(y_test, y_test_pred, beta=1))

Naive Bayes Training Dataset Confusion Matrix
 [[224  32]
 [  8 248]]
Naive Bayes Testing Dataset Confusion Matrix
 [[91 14]
 [ 3 19]]
Training Accuracy: 0.9253731343283582
Testing Accuracy: 0.6909090909090909


### Error-Based Classification Models

In [ ]:
## create a Logistic Regression classifier
logreg = LogisticRegression(random_state= 10, penalty= 'l1', solver='liblinear') ## l1 is lasso regression (https://www.statisticshowto.com/lasso-regression/) and liblinear (https://holypython.com/log-reg/logistic-regression-optimization-parameters/)
logreg = logreg.fit(X_train, y_train)

y_train_pred = logreg.predict(X_train)
y_test_pred = logreg.predict(X_test)

## display the confusion matrix 
print("Logistic Regression Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("Logistic Regression Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the F2 score for the training and testing datasets
print("Training Dataset F2 Score:", fbeta_score(y_train, y_train_pred, beta=1.0))
print("Testing Dataset F2 Score:", fbeta_score(y_test, y_test_pred, beta=1.0)) 

Logistic Regression Training Dataset Confusion Matrix
 [[244  12]
 [  0 256]]
Logistic Regression Testing Dataset Confusion Matrix
 [[95 10]
 [ 2 20]]
Training Dataset F2 Score: 0.9770992366412213
Testing Dataset F2 Score: 0.7692307692307692


In [ ]:
## use GridSearchCV to tune model hyperparameters 
svm_pipeline = Pipeline([('sampling', RandomOverSampler()), ('class', SVC())])

params = {'class__kernel': ['linear'], ## linear kernel is used here because feature coefficients are only available for the linear kernel
          'class__C': [0.5, 1.0, 10, 100],
          'class__gamma': ['scale', 'auto']}

gs_svm = GridSearchCV(svm_pipeline, params, scoring='f1', cv= 10)
gs_svm.fit(X_train, y_train)
print(gs_svm.best_params_)

## create a Support Vector Machine classifier
svm = SVC(kernel='linear', C=10) ## gamma= 'scale' is the default
svm = svm.fit(X_train, y_train)

y_train_pred = svm.predict(X_train)
y_test_pred = svm.predict(X_test)

## display the confusion matrix 
print("SVM Training Dataset Confusion Matrix\n", confusion_matrix(y_train, y_train_pred))
print("SVM Testing Dataset Confusion Matrix\n", confusion_matrix(y_test, y_test_pred))

## report the F2 score for the training and testing datasets
print("Training Dataset F2 Score:", fbeta_score(y_train, y_train_pred, beta=1.0))
print("Testing Dataset F2 Score:", fbeta_score(y_test, y_test_pred, beta=1.0))

SVM Training Dataset Confusion Matrix
 [[245   8]
 [  0 253]]
SVM Testing Dataset Confusion Matrix
 [[106   2]
 [  4  15]]
Training Dataset F2 Score: 0.9844357976653696
Testing Dataset F2 Score: 0.8333333333333333


## Feature Importance

The random forest classifier was the best performing model, with a training accuracy of 0.9865 and a testing accuracy of 0.947.

In [ ]:
tree_estimator = RandomForestClassifier(random_state= 10, n_estimators= 100, criterion= 'gini')
tree_estimator.fit(X_train, y_train)

feats = {} # reference: https://machinelearningmastery.com/calculate-feature-importance-with-python/; https://stackoverflow.com/questions/41900387/mapping-column-names-to-random-forest-feature-importances
for feature, importance in zip(X.columns, tree_estimator.feature_importances_):
    feats[feature] = importance

importance_df = pd.DataFrame.from_dict(feats, orient='index', columns=['Feature Importance'])
importance_df = importance_df.sort_values(by= ['Feature Importance'], ascending= False)

print(importance_df.head(n=10))

                              Feature Importance
SeverityDiseaseHumans_None              0.152038
ChronicityHumans_Acute                  0.114404
SeverityDiseaseHumans_Deadly            0.081531
VirusGenus_Lyssavirus                   0.073241
VirusFamily_Rhabdoviridae               0.068231
ChronicityHumans_None                   0.063312
Total_transmission                      0.028674
HostOrder_Chiroptera                    0.023747
VirusGenus_Orthohepevirus               0.014934
VirusFamily_Paramyxoviridae             0.014515


In [ ]:
## fit a logit-link model using severity, chronicity, transmission, genome type, segmentation, and envelope
formula = 'HumanToHumanTransVirus ~ SeverityDiseaseHumans+ChronicityHumans+Total_transmission+Genome_General+Segmented+Envelope'

model = smf.glm(formula = formula, data=spillover_data, family= sm.families.Binomial())
result = model.fit()
print(result.summary())

                                       Generalized Linear Model Regression Results                                       
Dep. Variable:     ['HumanToHumanTransVirus[No]', 'HumanToHumanTransVirus[Yes]']   No. Observations:                  423
Model:                                                                       GLM   Df Residuals:                      413
Model Family:                                                           Binomial   Df Model:                            9
Link Function:                                                             logit   Scale:                          1.0000
Method:                                                                     IRLS   Log-Likelihood:                    nan
Date:                                                           Sat, 11 Dec 2021   Deviance:                          nan
Time:                                                                   00:44:28   Pearson chi2:                     165.
No. Iterations:         

/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:890: RuntimeWarning: invalid value encountered in true_divide
  n_endog_mu = self._clean((1. - endog) / (1. - mu))
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:942: RuntimeWarning: divide by zero encountered in true_divide
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu)) +
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: divide by zero encountered in log
  n * np.log(1 - mu)) * var_weights
/usr/local/lib/python3.7/dist-packages/statsmodels/genmod/families/family.py:943: RuntimeWarning: invalid value encountered in add
  n * np.log(1 - mu)) * var_weights


In [ ]:
## fit a logit-link model using viral genomic characteristics (genomic material, envelope, segmentation)
formula = 'HumanToHumanTransVirus ~ Genome_General+Segmented+Total_transmission'

model = smf.glm(formula = formula, data=spillover_data, family= sm.families.Binomial())
result = model.fit()
print(result.summary())

                                       Generalized Linear Model Regression Results                                       
Dep. Variable:     ['HumanToHumanTransVirus[No]', 'HumanToHumanTransVirus[Yes]']   No. Observations:                  423
Model:                                                                       GLM   Df Residuals:                      419
Model Family:                                                           Binomial   Df Model:                            3
Link Function:                                                             logit   Scale:                          1.0000
Method:                                                                     IRLS   Log-Likelihood:                -168.09
Date:                                                           Sat, 11 Dec 2021   Deviance:                       336.19
Time:                                                                   00:44:31   Pearson chi2:                     412.
No. Iterations:         